In [ ]:
! pip install -qU llama-index-llms-gemini llama-index llama-index-embeddings-gemini

In [ ]:
!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2

In [ ]:
!pip install google-api-python-client

In [ ]:
!pip install spotipy

In [ ]:
# main.py

from config import GOOGLE_API_KEY, SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET, YOUTUBE_API_KEY

# Use the keys as needed
# print(GOOGLE_API_KEY)
# print(SPOTIFY_CLIENT_ID)
# print(SPOTIFY_CLIENT_SECRET)
# print(YOUTUBE_API_KEY)


In [ ]:

from llama_index.llms.gemini import Gemini
llm = Gemini(
    model="models/gemini-1.5-flash",
    api_key=GOOGLE_API_KEY,
)

In [ ]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import datetime

# Step 1: Define the required scope
SCOPES = ['https://www.googleapis.com/auth/calendar']

# Step 2: Launch the OAuth flow
flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
creds = flow.run_local_server(port=0, open_browser=False)

# Step 3: Build the Calendar API service
service = build('calendar', 'v3', credentials=creds)


In [ ]:
# Input section
topic = input("🎥 Enter the topic you want to explore (e.g., 'learn chess tactics'): ")
preferred_duration_minutes = int(input("⏱️ What is your preferred event duration (in minutes, e.g., 15): "))
reminders_on = input("Would you like a reminder 15 minutes before each event starts?")

# Preview the inputs
print("\n✅ You entered:")
print(f"Topic: {topic}")
print(f"Preferred Duration: {preferred_duration_minutes} minutes")
print(f"Reminders On(yes/no): {reminders_on}")


In [ ]:
# Youtube API connection to pull the relevant output for user request.

from googleapiclient.discovery import build
from datetime import datetime
import os

# Determine YouTube duration category
if preferred_duration_minutes < 4:
    duration_category = 'short'
elif preferred_duration_minutes <= 20:
    duration_category = 'medium'
else:
    duration_category = 'long'

# Initialize YouTube API
YOUTUBE_API_KEY = YOUTUBE_API_KEY
youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)

# Search videos
youtube_list = youtube.search().list(
    q=topic,
    part='snippet',
    type='video',
    maxResults=10,
    videoDuration=duration_category,
    order='relevance'
).execute()




In [ ]:
import os
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

# Set these from your Spotify Developer Dashboard
os.environ['SPOTIPY_CLIENT_ID'] = SPOTIFY_CLIENT_ID
os.environ['SPOTIPY_CLIENT_SECRET'] = SPOTIFY_CLIENT_SECRET
# print(os.environ['SPOTIPY_CLIENT_SECRET'])
# Auth and client
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

# Search for podcast episodes based on topic
results = sp.search(q=topic, type='episode', limit=10)

# Convert preferred minutes to seconds
max_duration_sec = preferred_duration_minutes * 60

# Filter relevant episodes
spotify_list = [
    ep for ep in results['episodes']['items']
    if ep['duration_ms'] <= max_duration_sec * 1000
]



In [ ]:
# Prompt Gemini to create a nice list for the resulting spotify and youtube results
summarized_list_prompt = f"""spotify_list = {spotify_list}
youtube_list = {youtube_list}
For the above spotify and youtube lists give a formatted version of results that only lists the title and duration for each.
"""

summarized_list = llm.complete(summarized_list_prompt).text.strip()

print(f"🔍 These were the results found: \n{summarized_list}")



In [ ]:
def list_events(start_datetime, end_datetime):
    events_result = service.events().list(
        calendarId='primary',
        timeMin=start_datetime.isoformat() + 'Z',
        timeMax=end_datetime.isoformat() + 'Z',
        singleEvents=True,
        orderBy='startTime'
    ).execute()

    events = events_result.get('items', [])
    return events

def get_user_timezone():
    tz_setting = service.settings().get(setting='timezone').execute()
    return tz_setting['value']

def insert_event(event):
  event = service.events().insert(calendarId='primary', body=event).execute()
  return event

In [ ]:
import datetime
userEvents = list_events(datetime.datetime.utcnow(), datetime.datetime.utcnow() + datetime.timedelta(days=20))

In [ ]:
import re
import json

def extract_json_from_text(text):
    """
    Extracts the first JSON array found in the given text.

    Parameters:
        text (str): The text containing the JSON.

    Returns:
        list or None: Parsed JSON object (list of dicts) if found, otherwise None.
    """
    try:
        json_match = re.search(r'```json\s*(\[\s*{.*?}\s*\])\s*```', text, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
            return json.loads(json_str)
        else:
            print("No JSON found in the text.")
            return None
    except json.JSONDecodeError as e:
        print(f"JSON decoding error: {e}")
        return None


In [ ]:


scheduling_prompt = f"""
This is the user's current schedule: {userEvents}

These are the spotify results: {spotify_list}

These are the youtube results: {youtube_list}

The user's timezone is {get_user_timezone}

The user's reminder preference is {reminders_on}

The user's calendar id is 'primary'.

Create an json array of scheduled events based on the user’s schedule and the Spotify and Youtube results. 
There should only be either one video or one podcast per event.  There should be a max of 2 events per day, one in the morning, one at night.

Give the result in a json format like this:

[
  {{
    'summary': 'Topic: Video or Podcast Title',
    'description': '
    Spotify/Youtube URL: Url here
    Description: Short video or podcast description here if available
    ',
    'start': {{
      'dateTime': '2015-05-28T09:00:00-07:00',
      'timeZone': 'America/Los_Angeles',
    }},
    'end': {{
      'dateTime': '2015-05-28T17:00:00-07:00',
      'timeZone': 'America/Los_Angeles',
    }},
    'reminders': {{
      'useDefault': False,
      'overrides': [
        {{'method': 'popup', 'minutes': 15}},
      ],
    }},
  }}
]

Create a list of events that will be scheduled into Google Calendar based on only the Spotify and Youtube results.  
If it's a Spotify url the description should start with Spotify URL: and if it's a Youtube url it should start with Youtube URL:.
Try to include as many of the Spotify and Youtube results as possible.
Do not include any of the users own events in the resulting json array.
"""

scheduling_list = extract_json_from_text(llm.complete(scheduling_prompt).text.strip())

print(f"Scheduling List: {scheduling_list}")
confirmation_prompt = f"""
Summaries these events that will be scheduled into Google Calendar into a readable format for confirmation by the user.

{scheduling_list}
"""

confirmation = llm.complete(confirmation_prompt).text.strip()
print(f"Scheduling List: {confirmation}")

confirmation_input = input(f"These are the events that will be scheduled, confirm submission?  (y/n)")

if confirmation_input == "y":
  for entry in scheduling_list:
    insert_event(entry)
else:
  print("Then guess you have to start the program over because I am too lazy to return the function or put this in a loop.  Goodbye.")



